<a href="https://colab.research.google.com/github/Athugodage/Hyperskill_transliteration/blob/main/transliteration.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install huggingface_hub
!pip install transformers datasets evaluation sacrebleu Sentencepiece

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 200.1/200.1 kB 5.0 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 49.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 468.7/468.7 kB 37.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 118.9/118.9 kB 15.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 58.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 53.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.9/132.9 kB 16.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 7.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 74.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
from huggingface_hub import notebook_login

notebook_login()

Token is valid.
Your token has been saved in your configured git credential helpers (store).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [ ]:
from datasets import load_dataset

In [ ]:



dataset = load_dataset('csv', 
                       data_files={'train': ['train.csv'],
                                   'test': ['test.csv']},
                       encoding='utf8',
                       index_col = False,
                       header=None,
                       sep=','
                      )

Extracting data files:   0%|          | 0/2 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

Dataset csv downloaded and prepared to /root/.cache/huggingface/datasets/csv/default-866682cf83f00415/0.0.0/6954658bab30a358235fa864b05cf819af0e179325c740e4bc853bcc7ec513e1. Subsequent calls will reuse this data.


  0%|          | 0/2 [00:00<?, ?it/s]

In [ ]:
dataset.push_to_hub('marcus2000/polish_names')

Pushing dataset shards to the dataset hub:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

Upload 1 LFS files:   0%|          | 0/1 [00:00<?, ?it/s]

Deleting unused files from dataset repository:   0%|          | 0/1 [00:00<?, ?it/s]

Pushing dataset shards to the dataset hub:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

Upload 1 LFS files:   0%|          | 0/1 [00:00<?, ?it/s]

In [ ]:
dataset = load_dataset('marcus2000/polish_names')

Extracting data files:   0%|          | 0/2 [00:00<?, ?it/s]

Generating test split:   0%|          | 0/64 [00:00<?, ? examples/s]

Generating train split:   0%|          | 0/572 [00:00<?, ? examples/s]

Dataset parquet downloaded and prepared to /root/.cache/huggingface/datasets/marcus2000___parquet/marcus2000--polish_names-fa6547915a779d8a/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec. Subsequent calls will reuse this data.


  0%|          | 0/2 [00:00<?, ?it/s]

In [ ]:
from transformers import AutoTokenizer, AutoModel, AutoConfig


tokenizer = AutoTokenizer.from_pretrained("t5-base", use_fast=True)

/usr/local/lib/python3.9/dist-packages/transformers/models/t5/tokenization_t5_fast.py:155: FutureWarning: This tokenizer was incorrectly instantiated with a model max length of 512 which will be corrected in Transformers v5.
For now, this behavior is kept to avoid breaking backwards compatibility when padding/encoding with `truncation is True`.
- Be aware that you SHOULD NOT rely on t5-base automatically truncating your input to 512 when padding/encoding.
- If you want to encode/pad to sequences longer than 512 you can either instantiate this tokenizer with `model_max_length` or pass `max_length` when encoding/padding.
- To avoid this warning, please instantiate this tokenizer with `model_max_length` set to your preferred value.
  warnings.warn(


In [ ]:

from transformers import AutoModelForSeq2SeqLM, DataCollatorForSeq2Seq, Seq2SeqTrainingArguments, Seq2SeqTrainer

model_hf = AutoModelForSeq2SeqLM.from_pretrained("t5-base")

In [ ]:
dataset

DatasetDict({
    train: Dataset({
        features: ['0', '1'],
        num_rows: 637
    })
})

In [ ]:
max_input_length = 10
max_target_length = 10


def preprocess_function(examples):
    inputs = [ex for ex in examples['0']]  # 0 - Текст
    targets = [ex for ex in examples['1']]  # 1 - Комментарий
    model_inputs = tokenizer(inputs, 
                             max_length=max_input_length,
                             truncation=True, padding=True)

    # Setup the tokenizer for targets
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(targets, max_length=max_target_length, truncation=True, padding=True)

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs
     

In [ ]:
tokenized_datasets = dataset.map(preprocess_function, batched=True)
     

Map:   0%|          | 0/64 [00:00<?, ? examples/s]

/usr/local/lib/python3.9/dist-packages/transformers/tokenization_utils_base.py:3596: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


Map:   0%|          | 0/572 [00:00<?, ? examples/s]

In [ ]:
from transformers import AutoModelForSeq2SeqLM, DataCollatorForSeq2Seq, Seq2SeqTrainingArguments, Seq2SeqTrainer

batch_size = 1

args = Seq2SeqTrainingArguments("polish_transliterator1",
    evaluation_strategy = "epoch",
    learning_rate=3e-6,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    weight_decay=0.01,
    save_total_limit=3,
    num_train_epochs=30,
    push_to_hub=True,
    predict_with_generate=True,
    fp16=True
)
    

In [ ]:
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model_hf)

In [ ]:
from datasets import load_metric


metric = load_metric("bleu")

<ipython-input-12-f73f02594bd2>:4: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  metric = load_metric("bleu")


In [ ]:
import numpy as np

def postprocess_text(preds, labels):
    preds = [pred.strip() for pred in preds]
    labels = [[label.strip()] for label in labels]

    return preds, labels

def compute_metrics(eval_preds):
    preds, labels = eval_preds
    if isinstance(preds, tuple):
        preds = preds[0]
    decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)

    # Replace -100 in the labels as we can't decode them.
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    # Some simple post-processing
    decoded_preds, decoded_labels = postprocess_text(decoded_preds, decoded_labels)

    result = metric.compute(predictions=decoded_preds, references=decoded_labels)
    result = {"bleu": result["score"]}

    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in preds]
    result["gen_len"] = np.mean(prediction_lens)
    result = {k: round(v, 4) for k, v in result.items()}
    return result

In [ ]:
trainer = Seq2SeqTrainer(
    model_hf,
    args,
    train_dataset=tokenized_datasets['train'],
    eval_dataset=tokenized_datasets['test'],
    data_collator=data_collator,
    tokenizer=tokenizer)

/content/polish_transliterator1 is already a clone of https://huggingface.co/marcus2000/polish_transliterator1. Make sure you pull the latest changes with `repo.git_pull()`.


In [ ]:
trainer.train()

/usr/local/lib/python3.9/dist-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch,Training Loss,Validation Loss
1,3.223000,2.203234
2,2.606900,1.578920
3,2.141400,1.290769
4,1.781000,1.056830
5,1.509400,0.983167
6,1.380800,0.932292
7,1.084100,0.900743
8,0.982300,0.875104
9,0.911400,0.848503
10,0.901300,0.826496


Epoch,Training Loss,Validation Loss
1,3.223000,2.203234
2,2.606900,1.578920
3,2.141400,1.290769
4,1.781000,1.056830
5,1.509400,0.983167
6,1.380800,0.932292
7,1.084100,0.900743
8,0.982300,0.875104
9,0.911400,0.848503
10,0.901300,0.826496


TrainOutput(global_step=17160, training_loss=0.9929249659006968, metrics={'train_runtime': 5984.8237, 'train_samples_per_second': 2.867, 'train_steps_per_second': 2.867, 'total_flos': 204096043008000.0, 'train_loss': 0.9929249659006968, 'epoch': 30.0})

In [ ]:
trainer.push_to_hub()

Upload file pytorch_model.bin:   0%|          | 1.00/850M [00:00<?, ?B/s]

Upload file runs/Apr13_19-51-00_6b9182fabfd8/events.out.tfevents.1681416082.6b9182fabfd8.514.12:   0%|        …

To https://huggingface.co/marcus2000/polish_transliterator
   aee2746..b50c4f3  main -> main

   aee2746..b50c4f3  main -> main

To https://huggingface.co/marcus2000/polish_transliterator
   b50c4f3..9722e7c  main -> main

   b50c4f3..9722e7c  main -> main



'https://huggingface.co/marcus2000/polish_transliterator/commit/b50c4f329c7d4d2b1c287cedfd240231a9d0e73f'

In [ ]:
df.head()

,Polish,English
0,Adam,Adam
1,Adrian,Adrian
2,Albert,Albert
3,Albin,Albin
4,Aleksander,Alexander


In [ ]:
from transformers import pipeline

model = pipeline('text2text-generation', model='marcus2000/polish_transliterator')

In [ ]:
for name in dataset['test']['0']:
  print('Polish: ', name, 'English: ', model(name)[0]['generated_text'])

Polish:  Żelisław English:  elisaw
Polish:  Wyspy Salomona English:  Wyspy Salomona
Polish:  Ginter English:  Ginter
Polish:  Karina English:  Karina
Polish:  Leo English:  Leo
Polish:  Słowacja English:  Sowacja
Polish:  Dobrosław English:  Dobrosiaw
Polish:  Halina English:  Halina
Polish:  Nowa Zelandia English:  Zeland
Polish:  Romuald English:  Romuald
Polish:  Liban English:  Libanon
Polish:  Witold English:  Witold
Polish:  Izrael English:  Izrael
Polish:  Słowenia English:  Sowenia
Polish:  Korea Północna English:  Korea
Polish:  Somalia English:  Somalia
Polish:  Gwinea English:  Gwinea
Polish:  Albert English:  Albert
Polish:  Aleksandra English:  Aleksandra
Polish:  Rozalia English:  Rozalia
Polish:  Turcja English:  Turcja
Polish:  Bogusław English:  Bogus
Polish:  Maria English:  Maria
Polish:  Angela English:  Angela
Polish:  Antonina English:  Antonina
Polish:  Renata English:  Renata
Polish:  Janusz English:  Janusz
Polish:  Egipt English:  Egipt
Polish:  Ireneusz Engli

In [ ]:
model('Aleksander')

[{'generated_text': 'Aleksander'}]